In [ ]:
# To import all the necessary packages 
# install -r requirements.txt
import whisper
from gtts import gTTS
from transformers import BlipProcessor, BlipForConditionalGeneration
from datasets import load_dataset
from PIL import Image
import torch
import os
import tempfile
from IPython.display import Audio
from deep_translator import GoogleTranslator 
from langdetect import detect


In [ ]:
# %pip install git+https://github.com/huggingface/transformers
%pip install deep-translator

In [ ]:
import whisper
whisper_model = whisper.load_model("base")
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
# blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
# blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)


In [ ]:

# Load datasetimport pandas as pd
from PIL import Image
import os
import pandas as pd

# Load the CSV
df = pd.read_csv("Cleaned_Temples_Significance.csv")

# Folder where images are stored
image_folder = "images"

# Load images using PIL and add to dataframe
def load_image(row):
    image_path = os.path.join(image_folder, row['image_url'])
    return Image.open(image_path)

# Apply the function to create a new column with PIL images
df['image_pil'] = df.apply(load_image, axis=1)

# Convert dataset to dictionary
temple_info = {row["name"]: row["image_url"] for _, row in df.iterrows()}

print(temple_info)

In [ ]:

# from sentence_transformers import SentenceTransformer, util


# # Load Temple Dataset
# df = pd.read_csv("Dataset1.csv")  # must contain 'name', 'description'

# # Encode all descriptions
# embedder = SentenceTransformer('all-MiniLM-L6-v2')
# df['description_embedding'] = df['description'].apply(lambda desc: embedder.encode(desc, convert_to_tensor=True))

# def get_caption(image_path):
#     raw_image = Image.open(image_path).convert('RGB')
#     inputs = blip_processor(raw_image, return_tensors="pt").to(blip_model .device)
#     out = blip_model .generate(**inputs)
#     return blip_processor.decode(out[0], skip_special_tokens=True)

# def identify_temple_with_blip(image_path):
#     caption = get_caption(image_path)
#     caption_embed = embedder.encode(caption, convert_to_tensor=True)

#     # Semantic similarity
#     similarities = [util.pytorch_cos_sim(caption_embed, emb)[0][0].item() for emb in df['description_embedding']]
#     best_index = similarities.index(max(similarities))
    
#     return {
#         "caption": caption,
#         "name": df.iloc[best_index]['name'],
#         "description": df.iloc[best_index]['description']
#     }

# # 🔍 Example
# result = identify_temple_with_blip("D:\Heritage_chatbot\images\image6.jfif")
# print("🏯 Temple Name:", result['name'])
# print("📖 Description:", result['description'])
# print("🖼️ Caption:", result['caption'])


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

In [ ]:

# import numpy as np
# #Step 2: Load your temple dataset ===
# df = pd.read_csv("Cleaned_Temples_Significance.csv")  # contains columns: name, description, image_url
# image_folder = "images"

# def extract_features(pil_img):
#     try:
#         inputs = blip_processor(images=pil_img, return_tensors="pt").to(device)
#         with torch.no_grad():
#             outputs = blip_model.vision_model(**inputs)
#             features = outputs.pooler_output.squeeze(0)
#         return features
#     except Exception as e:
#         print(f"Feature extraction failed: {e}")
#         return None


# feature_vectors = []
# names = []
# descriptions = []

# for _, row in df.iterrows():
#     image_path = os.path.join(image_folder, row['image_url'])
#     try:
#         pil_img = Image.open(image_path).convert("RGB")
#         features = extract_features(pil_img)
#         if features is not None:
#             feature_vectors.append(features.cpu().numpy())
#             names.append(row['name'])
#             descriptions.append(row['description'])
#     except Exception as e:
#         print(f"Error processing image {image_path}: {e}")

# feature_matrix = np.array(feature_vectors)    

# # === Step 4: Function to match uploaded image ===
# def identify_temple(pil_img):
#     input_feature = extract_features(pil_img)
#     if input_feature is None:
#         return "Unknown Temple", "Feature extraction failed."

#     input_vector = input_feature.unsqueeze(0).cpu().numpy()

#     if feature_matrix.ndim == 1:
#         feature_matrix = feature_matrix.reshape(1, -1)

#     if feature_matrix.size == 0:
#         return "Unknown", "No reference data available."

#     similarities = cosine_similarity(input_vector, feature_matrix)
#     best_match_idx = similarities.argmax()

#     return names[best_match_idx], descriptions[best_match_idx]
# # === Step 5: Example usage ===
# if __name__ == "__main__":
#     # uploaded_image_path = "d:\Heritage_chatbot\sample_image.jfif"  # Replace with your test image
#     img = Image.open("D:/Heritage_chatbot/sample_image.jfif").convert("RGB")
#     name, desc = identify_temple(img)
#     print(f"\n🏛️ Temple Identified: {name}\n📖 Description: {desc}")

In [ ]:
def generate_caption(image):
    inputs = blip_processor(images=image, return_tensors="pt", max_length=100).to(device)
    out = blip_model.generate(**inputs)
    caption = blip_processor.decode(out[0], skip_special_tokens=True)
    # print(caption)
    return caption

In [ ]:

def transcribe(audio):
    
    #time.sleep(3)
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

    # detect the spoken language
    _, probs = whisper_model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(whisper_model, mel, options)
    return result.text
    print(result.text)
    

In [ ]:
%pip install webdriver-manager
%pip install selenium beautifulsoup4


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from langchain.docstore.document import Document

# ✅ Your list of websites and the class names to extract
sites_to_scrape = [
    {
        "url": "https://www.holidify.com/collections/historical-places-in-tamilnadu",
        "class_names": [
            "card-text",
            "card-heading",
            "travel-essentials-section mt-md-0",
            "slick-track",
            "textColor infoSpace",
            "card-body",
            "places-covered"
        ]
    },
    {
        "url": "https://www.southtourism.in/tamil-nadu-monuments-timings.php",
        "class_names": [
            "card-body border",
            "table table-bordered"
        ]
    },
    {
        "url": "https://www.tamilnadutourism.tn.gov.in/",
        "class_names": [
            "text-justify pb-0"
        ]
    }
]

# ✅ Set up headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=chrome_options)

# ✅ Store LangChain Documents
documents = []

# ✅ Scraping loop
for site in sites_to_scrape:
    url = site["url"]
    class_names = site["class_names"]

    print(f"\n🌐 Visiting: {url}")
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    for class_name in class_names:
        elements = soup.find_all(class_=class_name)
        if elements:
            print(f"\n✅ Found content for class: '{class_name}'")
            for i, el in enumerate(elements):
                text = el.get_text(separator="\n", strip=True)
                if text:
                    doc = Document(
                        page_content=text,
                        metadata={
                            "source": url,
                            "class": class_name,
                            "index": i
                        }
                    )
                    documents.append(doc)
        else:
            print(f"\n⚠️ No content found for class: '{class_name}'")

driver.quit()

# ✅ Done! You now have a list of LangChain Document objects
print(f"\n📄 Total documents extracted: {len(documents)}")
print("\n🔍 Sample content from first document:\n")
print(documents[0].page_content[:800])


In [ ]:
%pip install langchain

In [ ]:
documents

In [ ]:
from langchain.docstore.document import Document

cleaned_docs = [Document(page_content=clean_html(doc.page_content), metadata=doc.metadata) for doc in documents]


In [ ]:
cleaned_docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(cleaned_docs)
# documents[:5]

In [ ]:
len(documents)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = FAISS.from_documents(documents, embeddings)

In [ ]:

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# 🔧 Load your local model using pipeline
flan_pipeline = pipeline(
    "text2text-generation",
    model=flan_model,
    tokenizer=flan_tokenizer,
    max_length=2400,
    do_sample=False,  # deterministic output
    clean_up_tokenization_spaces=True
)



In [ ]:
flan_llm = HuggingFacePipeline(pipeline=flan_pipeline)

# Setup RAG chain with your retriever
rag_chain = RetrievalQA.from_chain_type(
    llm=flan_llm,
    retriever=db.as_retriever(),
    return_source_documents=True
)

In [ ]:
def detect_language(text):
    return detect(text)


def translate_to_english(text, src_lang):
    return GoogleTranslator(source=src_lang, target='en').translate(text)

def translate_from_english(text, target_lang):
    return GoogleTranslator(source='en', target=target_lang).translate(text)
    

In [ ]:
def generate_multilingual_response(user_input):
    detected_lang = detect_language(user_input)
    
    # Step 1: Translate to English if needed
    if detected_lang != 'en':
        english_input = translate_to_english(user_input, detected_lang)
    else:
        english_input = user_input

    response_en = rag_chain.invoke(english_input)
    result=response_en["result"]

    # Step 3: Translate back to user's language
    if detected_lang != "en":
        final_response = translate_from_english(result, detected_lang)
    else:
        final_response = result

    return final_response


In [ ]:
text="tell me about chennai"
print(detect_language(text))
print(generate_multilingual_response(text))


In [ ]:
 # Function: Generate voice
def text_to_speech(text):
    tts = gTTS(text)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        return fp.name


In [ ]:
#first gardio

import gradio as gr
def process_input(mode, image, audio, text_input):
    if mode == "Image":
        caption = generate_caption(image)
        question = "Tell me about this temple."
        full_prompt = f"{caption}\nQuestion: {question}"
        response = rag_chain.invoke({"query": full_prompt})
        answer = response["result"]

    elif mode == "Audio":
        full_prompt = transcribe(audio)
        response = rag_chain.invoke({"query": full_prompt})
        answer = response["result"]
        # answer = generate_multilingual_response(text_input)

    elif mode == "Text":
        answer = generate_multilingual_response(text_input)

    else:
        return "Invalid mode selected.", None

    tts_path = text_to_speech(answer)
    return answer, tts_path

def toggle_inputs(mode):
    return (
        gr.update(visible=(mode == "Image")),
        gr.update(visible=(mode == "Audio")),
        gr.update(visible=(mode == "Text"))
    )

with gr.Blocks() as ui:
    gr.Markdown("# 🛕 AI Heritage Guide")
    gr.Markdown("Choose how you want to interact: Image, Audio, or Text")

    mode = gr.Dropdown(
        ["Image", "Audio", "Text"],
        label="Choose Input Mode",
        value="Image"
    )

    image_input = gr.Image(label="Upload Temple Image", visible=True)
    audio_input = gr.Audio(label="Speak your question", visible=False,type="filepath")
    text_input = gr.Textbox(label="Type your question", visible=False)

    output_text = gr.Textbox(label="AI Response")
    output_audio = gr.Audio(label="Spoken Output")

    mode.change(fn=toggle_inputs, inputs=[mode], outputs=[image_input, audio_input, text_input])

    submit_btn = gr.Button("Ask")

    submit_btn.click(
        fn=process_input,
        inputs=[mode, image_input, audio_input, text_input],
        outputs=[output_text, output_audio]
    )

if __name__ == "__main__":
    ui.launch()